In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import time
import numpy as np

In [ ]:
###########################################
# Top 100 tags
###########################################
tree = ET.parse('../StatsData/Tags.xml')
root = tree.getroot()
chil = root.getchildren()
tagsDF = pd.DataFrame([ch.attrib for ch in chil])
tagsDF.Count=map(int, tagsDF.Count)

tagsDF=tagsDF.sort('Count',ascending=0)
tag_list = list(tagsDF.TagName)
UserTagList = tag_list[0:100] # top100 tags
#tagsDF[0:100]

In [ ]:
print "The top 100 tags are:"
print UserTagList

In [ ]:
###########################################
# load Users, Posts Data
###########################################
tree = ET.parse('../StatsData/Users.xml')
root = tree.getroot()
chil = root.getchildren()
UsersDF = pd.DataFrame([ch.attrib for ch in chil])

tree = ET.parse('../StatsData/Posts.xml')
root = tree.getroot()
chil = root.getchildren()
PostsDF = pd.DataFrame([ch.attrib for ch in chil])

In [ ]:
###########################################
# Find user score at each tag
###########################################
QID={}
User={}
questionDF = PostsDF[PostsDF['PostTypeId']=='1']

for tag in UserTagList:
    _tag = '<'+tag+'>'
    print tag
    t = time.time()
    iftag = list(_tag in s for s in questionDF.Tags.values.tolist())
    QID[_tag] = questionDF[iftag].Id.values.tolist()
    lst = list((PostsDF.OwnerUserId[a],int(PostsDF.Score[a]))  for a in PostsDF.index if (PostsDF.ParentId[a] in QID[_tag]))
    ps = pd.DataFrame(lst, columns=['OwnerUserId', 'Score'])
    User[tag] = ps.groupby('OwnerUserId').sum()
    print time.time()-t


In [ ]:
# save User DataFrame
import dill, os
if os.path.isfile('User.pickle'):
    os.remove('User.pickle')
dill.dump(User, open("../StatsData/User.pickle", "w"))

In [ ]:
#####################################
# join the dataframe of all tags
#####################################
left = User[UserTagList[0]].rename(columns={'Score': UserTagList[0]})
for tag in UserTagList[1:]:
    right = User[tag].rename(columns={'Score': tag})
    left = pd.merge(left, right, left_index=True, right_index=True, how='outer')
left=left.fillna(0)
if os.path.isfile('RawCount.pickle'):
    os.remove('RawCount.pickle')
dill.dump(left, open("../StatsData/RawCount.pickle", "w"))
left